In [1]:
# Installing the necessary libraries
!pip install sentence-transformers==2.2.0
!pip install datasets==2.0.0
!pip install torch==1.12.0

  Using cached torch-2.5.1-cp310-cp310-manylinux1_x86_64.whl.metadata (28 kB)
Using cached torch-2.5.1-cp310-cp310-manylinux1_x86_64.whl (906.4 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0
    Uninstalling torch-1.12.0:
      Successfully uninstalled torch-1.12.0
  Using cached torch-1.12.0-cp310-cp310-manylinux1_x86_64.whl.metadata (22 kB)
Using cached torch-1.12.0-cp310-cp310-manylinux1_x86_64.whl (776.3 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1
    Uninstalling torch-2.5.1:
      Successfully uninstalled torch-2.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.13.2 requires torch>=1.13.0, but you have torch 1.12.0 which is incompatible.
torchaudio 2.5.1+cu121 requires torch==2.5.1, but you have torch 1.12.0 which is incompatible.
torchvision 0.20.1+cu121 requires torch==2.5.1, 

In [3]:
# !pip uninstall huggingface-hub -y
# !pip install huggingface-hub==0.25.0

In [1]:
# Importing necessary modules
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util
import torch
import os

ModuleNotFoundError: No module named 'huggingface_hub.snapshot_download'

In [ ]:
# Load a sample from the Multi-News dataset and convert it to a pandas dataframe
dataset = load_dataset("multi_news", split="test")
df = dataset.to_pandas().sample(2000, random_state=42)

In [ ]:
# Load the SentenceTransformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Encode the summaries and store them as embeddings
df["embedding"] = list(model.encode(df["summary"].to_list(), show_progress_bar=True))
passage_embeddings = df["embedding"].to_list()

In [ ]:
def find_relevant_news(query):
    # Encode the query using the same model
    query_embedding = model.encode(query)

    # Calculate the cosine similarity between the query and passage embeddings
    similarities = util.cos_sim(query_embedding, passage_embeddings)

    # Get the indices of the top 3 most similar passages
    top_indices = torch.topk(similarities.flatten(), 3).indices

    # Retrieve the summaries of the top 3 passages and truncate them to 160 characters
    top_relevant_passages = [df.iloc[x.item()]["summary"][:160] + "..." for x in top_indices]

    return top_relevant_passages

In [ ]:
def clear_screen():
    os.system("clear")

In [ ]:
def interactive_search():
    print("Welcome to the Semantic News Search!\n")
    while True:
        print("Type in a topic you'd like to find articles about, and I'll do the searching! (Type 'exit' to quit)\n> ", end="")

        query = input().strip()

        if query.lower() == "exit":
            print("\nThanks for using the Semantic News Search! Have a great day!")
            break

        print("\n\tHere are 3 articles I found based on your query: \n")

        passages = find_relevant_news(query)
        for passage in passages:
            print("\n\t" + passage)

        input("\nPress Enter to continue searching...")
        clear_screen()

In [ ]:
# Start the interactive search
interactive_search()